In [3]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from io import StringIO 
import os
import numbers
import math

# --- Shared State ---
data = []
df = pd.DataFrame()
df_logs = pd.DataFrame()
keywords = ['Hallo ich bin Rodland die Rodel', 'Section', 'Voltage', 'position', 'Pos:', 'Velocity', 'vel', 'List', 'SPINDLE', 'speedlimit']
Voltage = Position = Section = Velocity = brake_power = speedlimit = np.nan
BT_members = BT_members_cnt = 0
start_time = None
sled_nr = ''
filename = ''
content = []
logdata = ''

#options for displaying the table
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)  


# --- defines ---
def start(change): #startup program
        global sled_name, filename, content
        file_info = upload_widget.value[0]
        decoded = bytes(file_info['content']).decode('utf-8', errors='ignore') #decode the uploaded file into readable data
        content = decoded.splitlines() #seperate each line from data
        filename.value = file_info['name']
        for line in content:
            line.replace('\n' , '')
        clean_lines(content)
        sled_name = grab_name(file_info)
        parsing(content)  
        draw_graph(range_slider.value)      

def clean_lines(lines): #clean up lines incase they accidentally get seperated 
    i = 0
    for x in lines:
        if lines[i] == '': #if line is empty delete it from index
            lines.pop(i)
            i -= 1
            continue
        if lines[i][0] != '[' or (lines[i][0] == '[' and lines[i][1].isalpha()): #if first character isnt a "[" or if the first character is a "[" but the second character isnt a number add this line to the previous one
            if i != 0: #ignore the first line
                lines[i-1] = lines [i-1] + lines[i]
                lines.pop(i)
                i -= 1
        i += 1
    return lines

def grab_name(file_info): #read name of the sled out of file name
    file_name = file_info["name"] #get the filename out of file_info
    file_name = file_name.split("_") #split the filename into parts
    for part in file_name: #go though all parts in filename and see if it matches the format Rxxx
        match = re.fullmatch(r'R(\d+)', part)
        if match:
            sled_number = int(match.group(1))
            sled_name = 'Sled#' + str(sled_number) + ':' #save sledname to be like this: Sled#014:
            return(sled_name)

def parsing(clean_lines): #getting values out of file
    global Voltage, Position, Section, Velocity, brake_power, BT_members, BT_members_cnt, start_time, df, data, speedlimit, df_logs
    data = []
    Voltage = Position = Section = Velocity = brake_power = speedlimit = np.nan
    BT_members = BT_members_cnt = 0
    start_time = None
    
    checks = 0
    
    for word in keywords: #for every word we need to check for
        for line in clean_lines: 
            if word in line:
                clean_line = line.replace('[', '').replace(']', '').replace(',', '').replace('\n', '') # remove all "[],\n"
                parts = re.split(r'[ =]', clean_line)
                
                if checks == 0 and parts[0].isdigit(): #set starttime for the first instance it finds "Hallo ich bin Rodland die Rodel"
                    start_time = int(parts[0])
                    continue
                
                try:
                    time = (int(parts[0]) - start_time) / 100 #for every other time value gets found make it relative and convert it to seconds from ms
                except:
                    continue

                indices = [i for i, item in enumerate(parts) if word in item] #get the location of word in line
                try:
                    match word:
                        case "position":
                            Position = next((parts[i + 1] for i in indices if sled_name in parts), np.nan) #only save position if sled name gets mentioned in line
                        case "Pos:":
                            Position = parts[indices[0] + 1] #get position value after word "Pos:" gets mentioned
                        case "Voltage":
                            Voltage = parts[indices[0] + 1].replace('V', '') #save voltage but without V
                        case "Section":
                            Section = parts[indices[0] + 1]
                        case "Velocity":
                            Velocity = parts[indices[0] + 1]
                        case "vel":
                            Velocity = parts[indices[0] + 1]
                        case "List": 
                            if 'Insert' in parts[indices[0]]: #if insert gets mentioned increase BT_Members
                                BT_members_cnt += 1
                            elif 'Remove' in parts[indices[0]]:#if remove gets mentioned decrese BT_Members
                                BT_members_cnt -= 1
                            BT_members = BT_members_cnt
                        case "SPINDLE": #when SPINDLE gets mentioned save value as brake_power after pos gets mentioned
                            for x, i in enumerate(parts):
                                if 'pos' in i:
                                    brake_power = parts[x + 1]
                                    break
                        case 'speedlimit':
                            print(parts)
                            print("test")
                            if is_float(parts[indices[0] + 1]):
                                speedlimit = parts[indices[0] + 1]
                except:
                    continue
                data.append({
                    'Time': time, 'Voltage': Voltage, 'Position': Position, 'Section': Section,
                    'Velocity': Velocity, 'BT_Members': BT_members, 'Brake_Power': brake_power, 'Speedlimit': speedlimit
                }) #add all values to table
                Section = Voltage = Position = Velocity = brake_power = BT_members = speedlimit = np.nan #reset value
        checks += 1
    # Create and clean DataFrame
    df = []
    df = pd.DataFrame(data)

    for col in ['Time', 'Voltage', 'Position', 'Section', 'Velocity', 'BT_Members', 'Brake_Power', 'Speedlimit']:
        df[col] = pd.to_numeric(df[col], errors='coerce') #convert values to int
    
    df = df.sort_values(by='Time').dropna(subset=['Time']) #sort all values by time and remove values that dont have times related to them
    df = df.groupby('Time', as_index=False).first() #group values with the same time

    print(content)
    
    fill_list() #fill up list when no values given

    #making dataframe for table at bottom
    print("####################")
    df_logs = pd.DataFrame(content, columns=['Log Entry'])
    df_logs['Timestamp'] = df_logs['Log Entry'].str.extract(r'\[(\d+)\]').astype(float)
    df_logs['Relative Time'] = (df_logs['Timestamp'] - start_time) / 100

    df_logs = df_logs.loc[:, ['Timestamp', 'Relative Time', 'Log Entry']]

    df_logs['Display Label'] = df_logs['Relative Time'].map(lambda x: f"{x:.2f}") + ' — ' + df_logs['Log Entry']

    multi_select.options = df_logs['Display Label']


def set_range_slider(change): #change the max value of sliders when file gets uploaded
    global df

        #slider for graph 1
    range_slider.max = ((df['Time'].max() + 100) // 100) * 100
    range_slider.value = 0, ((df['Time'].max() + 100) // 100) * 100
#    print(((df['Time'].max() + 100) // 100) * 100)

        #slider for graph 2
    range_slider_2.max = ((df['Position'].max() + 100) // 100) * 100
    range_slider_2.value = 0, ((df['Position'].max() + 100) //100) * 100
#    print(((df['Position'].max() + 100) // 100) * 100)

    
                    
def draw_graph(change): #draw both graphs
    with graph_1: #draw graph 1(Werte zu Zeit)


        
        clear_output(wait=True)
        my_min, my_max = range_slider.value #get min and max from slider
        fig, ax1 = plt.subplots(figsize=(25, 5)) #set size of graph

        # Base axis: Voltage
        ax1.set_xlim(my_min, my_max) #set min and max from slider
        ax1.set_xlabel("Time (s)", fontsize=18)
        ax1.grid(True)
        ax1.xaxis.set_major_locator(MaxNLocator(integer=True, nbins=20))# set ammount of vertical grid lines
        
        axes_used = [ax1]

        if voltage_toggle.value: #if toggle isnt off
            filtered = df[df['Voltage'].notna()] #make new list that leaves out empty entries
            line1, = ax1.plot(filtered['Time'], filtered['Voltage'], color='tab:blue', label='Voltage (V)') #draw graph
            ax1.set_ylabel('Voltage (V)', color='tab:blue', fontsize=18) #add the y axis
            ax1.tick_params(axis='y', labelcolor='tab:blue') #modify the y axis to match the color
#            legend_lines.append(line1)
#            legend_labels.append(line1.get_label())

        # Second axis: Position
        if position_toggle.value:
            ax2 = ax1.twinx() #copy settings of main axis
            ax2.spines['right'].set_position(('axes', 1.1)) #set axis to the right side
            filtered = df[df['Position'].notna()]
            line2, = ax2.step(filtered['Time'], filtered['Position'], where='post', color='tab:orange', label='Position (m)')
            ax2.set_ylabel('Position (m)', color='tab:orange', fontsize=18)
            ax2.tick_params(axis='y', labelcolor='tab:orange')
#            legend_lines.append(line2)
#            legend_labels.append(line2.get_label())
            axes_used.append(ax2)

        # Third axis: Section
        if section_toggle.value:
            ax3 = ax1.twinx()
            ax3.spines['right'].set_position(('axes', 1.2))
            filtered = df[df['Section'].notna()]
            line3, = ax3.step(filtered['Time'], filtered['Section'], where='post', color='tab:green', label='Section')
            ax3.set_ylabel('Section', color='tab:green', fontsize=18)
            ax3.tick_params(axis='y', labelcolor='tab:green')
#            legend_lines.append(line3)
#            legend_labels.append(line3.get_label())
            axes_used.append(ax3)

        # Fourth axis: Velocity
        if velocity_toggle.value:
            ax4 = ax1.twinx()
            ax4.spines['right'].set_position(('axes', 1.3))
            filtered = df[df['Velocity'].notna()]
            line4, = ax4.step(filtered['Time'], filtered['Velocity'], where='post', color='tab:red', label='Velocity (km/h)')
            if speedlimit_toggle.value:
                line7, = ax4.step(filtered['Time'], filtered['Speedlimit'], where='post', color='darkred')
            ax4.set_ylabel('Velocity (km/h)', color='tab:red', fontsize=18)
            ax4.tick_params(axis='y', labelcolor='tab:red')
#            legend_lines.append(line4)
#            legend_labels.append(line4.get_label())
            axes_used.append(ax4)

        # Fith axis: BT_Members
        if BT_members_toggle.value:
            ax5 = ax1.twinx()
            ax5.spines['right'].set_position(('axes', 1.3))
            filtered = df[df['BT_Members'].notna()]
            line5, = ax5.step(filtered['Time'], filtered['BT_Members'], where='post', color='tab:pink', label='BT_Members')
            ax5.set_ylabel('BT_Members', color='tab:pink', fontsize=18)
            ax5.tick_params(axis='y', labelcolor='tab:pink')
#            legend_lines.append(line5)
#            legend_labels.append(line5.get_label())
            axes_used.append(ax5)

        # Sixth axis: Brake Power
        if brake_power_toggle.value:
            ax6 = ax1.twinx()
            ax6.spines['right'].set_position(('axes', 1.3))
            filtered = df[df['Brake_Power'].notna()]
            line6, = ax6.step(filtered['Time'], filtered['Brake_Power'], where='post', color='tab:purple', label='Brake_Power')
            ax6.set_ylabel('Brake_Power', color='tab:purple', fontsize=18)
            ax6.tick_params(axis='y', labelcolor='tab:purple')
#            legend_lines.append(line6)
#            legend_labels.append(line6.get_label())
            axes_used.append(ax6)


        
        # Adjust axes to prevent label overlap
        for i, axis in enumerate(axes_used[1:], start=2):
            axis.spines['right'].set_visible(True)
            axis.spines['right'].set_position(('axes', .95 + 0.05 * (i - 1)))
            axis.spines['right'].set_linewidth(1)

        n = 0
        for i in df['Section']:
            if int(i) == 33:
                plt.axvline(x = df['Time'][n], color = 'yellow', linestyle = '--', linewidth = 2)  # x=2, red dashed line
            n += 1
        
        # set Title
        ax1.set_title("Sensor Values Over Time", fontsize=24)

        plt.tight_layout()
        plt.show() #show graphs
        
    with graph_2: #draw the second graph (Werte zu Position)

        clear_output(wait=True)
        fig, ax1 = plt.subplots(figsize=(25, 5))
#        legend_lines = []
#        legend_labels = []

        my_min_2, my_max_2 = range_slider_2.value

        ax1.set_xlim(my_min_2, my_max_2)
        ax1.set_xlabel("Position (m)", fontsize=18)
        ax1.grid(True)
        ax1.xaxis.set_major_locator(MaxNLocator(integer=True, nbins=20))

        axes_used = [ax1]

        if voltage_toggle_2.value:
            filtered = df[df['Voltage'].notna() & df['Position'].notna()]
            line1, = ax1.plot(filtered['Position'], filtered['Voltage'], color='tab:blue', label='Voltage (V)')
            ax1.set_ylabel('Voltage (V)', color='tab:blue', fontsize=18)
            ax1.tick_params(axis='y', labelcolor='tab:blue')
#            legend_lines.append(line1)
#            legend_labels.append(line1.get_label())

        if section_toggle_2.value:
            ax2 = ax1.twinx()
            ax2.spines['right'].set_position(('axes', 1.1))
            filtered = df[df['Section'].notna() & df['Position'].notna()]
            line2, = ax2.step(filtered['Position'], filtered['Section'], where='post', label='Section', color='tab:green')
            ax2.set_ylabel('Section', color='tab:green', fontsize=18)
            ax2.tick_params(axis='y', labelcolor='tab:green')
#            legend_lines.append(line2)
#            legend_labels.append(line2.get_label())
            axes_used.append(ax2)

        if velocity_toggle_2.value:
            ax3 = ax1.twinx()
            ax3.spines['right'].set_position(('axes', 1.2))
            filtered = df[df['Velocity'].notna() & df['Position'].notna()]
            line3, = ax3.step(filtered['Position'], filtered['Velocity'], where='post', label='Velocity (km/h)', color='tab:red')
            if speedlimit_toggle_2.value:
                line6, = ax3.step(filtered['Time'], filtered['Speedlimit'], where='post', color='darkred')
            ax3.set_ylabel('Velocity (km/h)', color='tab:red', fontsize=18)
            ax3.tick_params(axis='y', labelcolor='tab:red')
#            legend_lines.append(line3)
#            legend_labels.append(line3.get_label())
            axes_used.append(ax3)

        if BT_members_toggle_2.value:
            ax4 = ax1.twinx()
            ax4.spines['right'].set_position(('axes', 1.3))
            filtered = df[df['BT_Members'].notna() & df['Position'].notna()]
            line4, = ax4.step(filtered['Position'], filtered['BT_Members'], where='post', label='BT_Members', color='tab:pink')
            ax4.set_ylabel('BT_Members', color='tab:pink', fontsize=18)
            ax4.tick_params(axis='y', labelcolor='tab:pink')
#            legend_lines.append(line4)
#            legend_labels.append(line4.get_label())
            axes_used.append(ax4)

        if brake_power_toggle_2.value:
            ax5 = ax1.twinx()
            ax5.spines['right'].set_position(('axes', 1.4))
            filtered = df[df['Brake_Power'].notna() & df['Position'].notna()]
            line5, = ax5.step(filtered['Position'], filtered['Brake_Power'], where='post', label='Brake_Power', color='tab:purple')
            ax5.set_ylabel('Brake_Power', color='tab:purple', fontsize=18)
            ax5.tick_params(axis='y', labelcolor='tab:purple')
#            legend_lines.append(line5)
#            legend_labels.append(line5.get_label())
            axes_used.append(ax5)

        # Adjust y-axis spacing
        for i, axis in enumerate(axes_used[1:], start=2):
            axis.spines['right'].set_visible(True)
            axis.spines['right'].set_position(('axes', 0.95 + 0.05 * (i - 1)))
            axis.spines['right'].set_linewidth(1)

        n = 0
        for i in df['Section']:
            if int(i) == 33:
                plt.axvline(x = df['Position'][n], color = 'yellow', linestyle = '--', linewidth = 2)  # x=2, red dashed line
            n += 1
            
        ax1.set_title("Values by Position", fontsize=24)

        plt.tight_layout()
        plt.show()

    with df_output:
        filtered_df = df_logs[(df_logs['Relative Time'] >= my_min) & (df_logs['Relative Time'] <= my_max)]

        filtered_df = filtered_df.style\
            .format({
                'Timestamp': '{:.0f}',        # no decimals
                'Relative Time': '{:.2f}'     # two decimals
            })\
            .set_table_styles([
                {'selector': 'th', 'props': [('text-align', 'left')]},  # header alignment
                {'selector': 'td', 'props': [('text-align', 'left')]}   # cell alignment
            ])

        
        
        clear_output()
        display(filtered_df)
#        print(df_logs)
        
def data_reset(change): #reset the graphs and data index if called
    global data
    data = []

    with graph_1:
        clear_output(wait=True)
        plt.close()

    with graph_2:
        clear_output(wait=True)
        plt.close()

    draw_graph(change)

def fill_list(): # fill up the empty entries in the list of data
    global df

#    print(df)
    for name in df:
#        print(name)
        name_old = 0
        n = 0
        if name != 'Voltage' or name != 'Brake_Power'or name != 'Section':
            for i in df[name]:
    #            print(i)
                if math.isnan(i):
                    i = name_old
    #                print(i)
                    df.loc[n, name] = i
                else:
                    name_old = i
                n += 1
                #print(i)
        n = 0
        for i in df['Section']:
            if i == 0:
                df[n, 'Position']= np.nan
            n += 1
                
    #print(df)

def is_float(value): #check if value is not a float
    try:
        float(value)
        return True
    except ValueError:
        return False



def update_popup(change):
    popup_box.clear_output()
    if change['new']:
        with popup_box:
            display(HTML("""
            <div style="
                position: fixed;
                top: 60px;
                right: 30px;
                z-index: 9999;
                background-color: #fefefe;
                border: 2px solid #ddd;
                border-radius: 10px;
                padding: 20px;
                width: 300px;
                box-shadow: 0 0 15px rgba(0,0,0,0.2);
                font-family: sans-serif;
            ">
                <style>
                table {
                  font-family: arial, sans-serif;
                  border-collapse: collapse;
                  width: 100%;
                }
                
                td, th {
                  border: 1px solid #dddddd;
                  text-align: left;
                  padding: 8px;
                }
                
                tr:nth-child(even) {
                  background-color: #dddddd;
                }
                </style>
                </head>
                <body>
                
                <h1>Info</h1>

                <h3> Begriffe: </h3>

                </p> <b> Time </b> – Zeit nach Start der Logdaten aufnahme </p>
                </p> <b> Voltage </b> – Die Spannung der Batterie der Rodel </p>
                </p> <b> Velocity </b> – Geschwindigkeit mit der sich die Rodel bewegt </p>
                </p> <b> Position </b> – Wo sich die Position auf der Strecke befindet </p>
                </p> <b> Section </b> – In welchem Abschnitt sich die Rodel befindet </p>
                </p> <b> BT_Members </b> – Anzahl an Bluetooth Teilnehmern die mit der Rodel verbunden sind </p>
                </p> <b> Brake_Power </b> – Wie sehr die Bremsen geschlossen sind </p>

                <h3> Section info: </h3>
                                
                <table>
                  <tr>
                    <th>Sektion</th>
                    <th style="width:70%">Definition</th>
                  </tr>
                  <tr>
                    <td>0</td>
                    <td>Init</td>
                  </tr>
                  <tr>
                    <td>1</td>
                    <td>Entry</td>
                  </tr>
                  <tr>
                    <td>2</td>
                    <td>Function check</td>
                  </tr>
                  <tr>
                    <td>3</td>
                    <td>Bergfahrt</td>
                  </tr>
                  <tr>
                    <td>4</td>
                    <td>Talfahrt</td>
                  </tr>
                  <tr>
                    <td>5</td>
                    <td>Exit</td>
                  </tr>
                  <tr>
                    <td>6</td>
                    <td>End</td>
                  </tr>
                  <tr>
                    <td>8</td>
                    <td>Motorized mountain ride</td>
                  </tr>
                  <tr>
                    <td>9</td>
                    <td>Motorized creep</td>
                  </tr>
                  <tr>
                    <td>10</td>
                    <td>Track brake</td>
                  </tr>
                  <tr>
                    <td>33</td>
                    <td>Error</td>
                  </tr>
                  <tr>
                    <td>35</td>
                    <td>Sim mode</td>
                  </tr>
                </table>
                
                </body>
            </div>
            """))
    
# --- Widgets ---
upload_widget = widgets.FileUpload(accept='.log', multiple=False, description='Select File')

title = widgets.HTML(
    value="<h1 style='color: black;'>Logdata analysis</h1>"
)

filename = widgets.Label(value='')

title_box = widgets.Box([title], layout=widgets.Layout(padding='0 20px', align_items='center'))
upload_box = widgets.Box([upload_widget], layout=widgets.Layout(padding='0 20px', align_items='center'))
filename_box = widgets.Box([filename], layout=widgets.Layout(align_items='center'))

# Toggle button to show/hide the popup
toggle_popup = widgets.ToggleButton(
    value=False,
    description="ℹ️ Info",
    tooltip="Click for help",
    button_style='info',
    layout=widgets.Layout(width='120px')
)

range_slider = widgets.IntRangeSlider(
    value=[0, 100],
    min=0,
    max=100,
    step=10,
    description='Range:',
    continuous_update=False,
    layout = widgets.Layout(width='30%', height='50px'))

range_slider_2 = widgets.IntRangeSlider(
    value=[0, 100],
    min=0,
    max=100,
    step=10,
    description='Range:',
    continuous_update=False,
    layout = widgets.Layout(width='30%', height='50px'))

graph_1 = widgets.Output()

voltage_toggle = widgets.Checkbox(value=True, description='Voltage')
position_toggle = widgets.Checkbox(value=True, description='Position')
section_toggle = widgets.Checkbox(value=True, description='Section')
velocity_toggle = widgets.Checkbox(value=True, description='Velocity')
BT_members_toggle = widgets.Checkbox(value=True, description='BT_Members')
brake_power_toggle = widgets.Checkbox(value=True, description='Brake_Power')
speedlimit_toggle = widgets.Checkbox(value=True, description='Speedlimit')

graph_2 = widgets.Output()

voltage_toggle_2 = widgets.Checkbox(value=True, description='Voltage')
section_toggle_2 = widgets.Checkbox(value=True, description='Section')
velocity_toggle_2 = widgets.Checkbox(value=True, description='Velocity')
BT_members_toggle_2 = widgets.Checkbox(value=True, description='BT_Members')
brake_power_toggle_2 = widgets.Checkbox(value=True, description='Brake_Power')
speedlimit_toggle_2 = widgets.Checkbox(value=True, description='Speedlimit')

popup_box = widgets.Output()

# Accordion widget
logdata = widgets.HTML()

df_output = widgets.Output()

multi_select = widgets.SelectMultiple(
    options='',
    description='Select Logs:',
    layout=widgets.Layout(width='100%', height='250px')
)

show_logdata = widgets.Accordion(children=[df_output])
show_logdata.set_title(0, 'Show Logdata')


# --- layout boxes ---
header = widgets.HBox(
    [title_box, upload_box, filename_box, toggle_popup],
    layout=widgets.Layout(
        align_items='flex-end',
        padding='10px 0 20px 0'
    )
)

slider_panel = widgets.HBox(
    [range_slider],
    layout = widgets.Layout(justify_content='flex-start', padding='10px 0 20px 0'))

# -- Graph 1 --
toggle_box = widgets.VBox([
    voltage_toggle, 
    position_toggle, 
    section_toggle, 
    velocity_toggle, 
    BT_members_toggle, 
    brake_power_toggle,
    speedlimit_toggle,
])

toggle_box.layout = widgets.Layout(
    width='420px',
    padding='10px',
    margin='0 0 0 0',
    align_self='flex-start'
)

graph_1.layout = widgets.Layout(width='90%')

graph_panel_1 = widgets.HBox(
    [graph_1, toggle_box],
    layout=widgets.Layout(align_items='flex-start', width='100%')
)

# -- Graph 2 --
toggle_box_2 = widgets.VBox([
    voltage_toggle_2,
    section_toggle_2,
    velocity_toggle_2,
    BT_members_toggle_2,
    brake_power_toggle_2,
    speedlimit_toggle_2,
])

toggle_box_2.layout = widgets.Layout(
    width='420px',
    padding='10px',
    margin='0 0 0 0',
    align_self='flex-start'
)

graph_2.layout = widgets.Layout(width='90%')

graph_panel_2 = widgets.HBox(
    [graph_2, toggle_box_2],
    layout=widgets.Layout(align_items='flex-start', width='100%')
)

slider_panel_2 = widgets.HBox(
    [range_slider_2],
    layout = widgets.Layout(justify_content='flex-start', padding='10px 0 20px 0'))

ui = widgets.VBox(
    [header, graph_panel_1, slider_panel, graph_panel_2, slider_panel_2, show_logdata],
    layout=widgets.Layout(width='100%')
)

display(ui)
display(popup_box)


# --- make widgets responsive ---

for widget in [upload_widget]:
    widget.observe(start, names='value')

upload_widget.observe(set_range_slider, names='value')
upload_widget.observe(data_reset, names='value')

for widget in [range_slider, range_slider_2, voltage_toggle, position_toggle, section_toggle, velocity_toggle, BT_members_toggle, brake_power_toggle, speedlimit_toggle, voltage_toggle_2, section_toggle_2, velocity_toggle_2, BT_members_toggle_2, brake_power_toggle_2, speedlimit_toggle_2]:
    widget.observe(draw_graph, names='value')

toggle_popup.observe(update_popup, names='value')



Output()